# Projeto - Case VAI Shoes
##0 - Instalando e importando bibliotecas
Nesta etapa instalamos e importamos as principais bibliotecas utilizadas na construção do notebook.

In [ ]:
# Import das libs utilizadas

# Manipulação de dados
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
from textwrap import wrap
from statsmodels.tsa.stattools import acf, pacf #!pip install statsmodels
import statsmodels.api as sm
# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
# Modelagem - Clusterização
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
# Modelagem - Regressão
# Modelagem - Série Temporal
from keras.models import Sequential #!pip install keras e !pip install tensorflow
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import StandardScaler #!pip install sklearn
from keras.callbacks import ModelCheckpoint

## 1 - Análise Exploratório dos Dados

In [ ]:
#Qual é o total de vendas por mês?
vendas_mensais = pd.read_csv(r'/content/drive/tbl_vendas_mensais.csv', sep=';')
tbl_Prev = pd.read_excel(r'/content/drive/tbl_previsao_sop.xlsx')
tbl_Prev

,dt_sale,ds_product_line,qt_sale_sop_forecasting_m1,qt_sale_sop_forecasting_m2,qt_sale_sop_forecasting_m3
0,2019-01-01,Boot,167789.110866,138483.536174,187255.563187
1,2019-02-01,Boot,195084.541090,185449.923096,207299.477312
2,2019-03-01,Boot,182954.647242,191482.395658,209627.808009
3,2019-04-01,Boot,167830.431321,197193.228582,172977.038817
4,2019-05-01,Boot,138123.025353,148525.498923,153424.646889
...,...,...,...,...,...
203,2022-11-01,Sneaker,723277.398304,551401.031558,545363.400758
204,2022-12-01,Sneaker,531355.605663,446024.079283,382002.914204
205,2023-01-01,Sneaker,477247.619917,346502.729432,287765.688886
206,2023-02-01,Sneaker,417588.606878,380837.123169,319618.928379


_A base de dados vai de Jan/2019 a Mar/2023_

In [ ]:
vendas_mensais['vl_sale_price'] = vendas_mensais['vl_sale_price'].apply(lambda x: float(str(x).replace(',', '.')))

_Houve um possivel erro ao carregar os dados de Novembro de 2021, os dados estão duplicados mas com valores de vendas diferentes, para contornar esse problema, vamos utilizar a média entre os dois valores_

In [ ]:
vendas_mensais = vendas_mensais.groupby(['dt_sale','id_sku','ds_sku','ds_category','ds_product_line','ds_brand_segment','ds_tecnology']).mean().reset_index()
#sop = sop.groupby(['dt_sale','ds_product_line']).mean().reset_index()

_Algumas datas possuem formatação diferente, então para realizar seu tratamento, primeiro vamos transforma-las em data e depois mudar o formato com que elas são exibidas. Como os dados são mensais, vamos deixar a formatação com o primeiro dia do mês, o mês correspondente e o ano_

In [ ]:
vendas_mensais.head(2)

,dt_sale,id_sku,ds_sku,ds_category,ds_product_line,ds_brand_segment,ds_tecnology,qt_sale,vl_sale_price
0,2019-01-01 00:00:00,1,AeroJet,Aero Boot,Boot,Target,NanoMesh,37873.0,876388.4791
1,2019-01-01 00:00:00,2,FlyQuest,Aero Boot,Boot,Target,LiteWeave,32881.0,832876.3526


_Procurando por valores nulos, encontramos que a coluna "vl_sale_price" possui 24 valores nulos, para tentar justificar a presença desses valores vamos investigar para avaliar o tipo de tratamento que será utilizado

Coluna | Valores nulos
:---: | :---:
vl_sale_price | 24

_Encontramos que os dados vazios correspondem a 2 produtos: NYC Wave e NYC Sub. Que pertencem a mesma categoria, linha, marca e usam a mesma tecnologia. Possivelmente são 2 produtos que foram adicionados recentemente a base de dados da empresa (Abril de 2022)_

_Encontramos que os dados vazios correspondem a 2 produtos: NYC Wave e NYC Sub. Que pertencem a mesma categoria, linha, marca e usam a mesma tecnologia. Possivelmente são 2 produtos que foram adicionados recentemente a base de dados da empresa (Abril de 2022)_

In [ ]:
vendas_mensais[vendas_mensais.vl_sale_price.isnull()][['id_sku', 'ds_sku', 'ds_category', 'ds_product_line',
       'ds_brand_segment', 'ds_tecnology']].drop_duplicates()

,id_sku,ds_sku,ds_category,ds_product_line,ds_brand_segment,ds_tecnology
1409,32,Nyc Wave,NYCs Shoes,Shoes,Core,LiteWeave
1410,33,Nyc Sub,NYCs Shoes,Shoes,Core,LiteWeave


In [ ]:
gera_vl_vazios = vendas_mensais.\
  query('ds_category=="NYCs Shoes" and ds_product_line=="Shoes" and ds_brand_segment=="Core" and ds_tecnology=="LiteWeave"').\
  dropna()

gera_vl_vazios['unit_price'] = gera_vl_vazios['vl_sale_price']/gera_vl_vazios['qt_sale']

In [ ]:
px.line(gera_vl_vazios, x='dt_sale', y='unit_price')

_Como o preço desse conjunto permanece constante no periodo em que esses 2 produtos foram inclusos na base, vamos selecionar os dados que correspondem a média do preço dos produtos da mesma classe e utiliza-los para preencher os valores vazios_

In [ ]:
preco_novos_calcados = gera_vl_vazios.query("dt_sale >= '2022-04-01'").unit_price.mean()

In [ ]:
vendas_mensais['vl_sale_price'] = vendas_mensais.apply(lambda x: x['qt_sale'] * preco_novos_calcados if pd.isna(x['vl_sale_price']) else x['vl_sale_price'], axis=1)

_Outro problema observado na base de dados é a questão das classes **SNEARKER**  e **SNEAKERS**, que representam o mesmo grupo. Então para corrigir esse problema vamos uni-los_

In [ ]:
# Convert the 'dt_sale' column back to datetime objects before using the .dt accessor
vendas_mensais['dt_sale'] = pd.to_datetime(vendas_mensais['dt_sale'])

fig = px.bar(vendas_mensais.groupby(vendas_mensais['dt_sale'].dt.year).agg({'vl_sale_price':'sum'}).reset_index(),\
              x='dt_sale', y='vl_sale_price')

fig.update_layout(xaxis_title="Período",  title='Faturamento Anual', yaxis_title='Vendas')

fig.show()

In [ ]:
fig = px.bar(vendas_mensais.groupby(vendas_mensais['dt_sale'].dt.year).agg({'vl_sale_price':'sum'}).reset_index(),\
              x='dt_sale', y='vl_sale_price')

fig.update_layout(xaxis_title="Período",  title='Faturamento Anual', yaxis_title='Vendas')

fig.show()

In [ ]:
#Analisando a série temporal como um todo
fig = px.line(vendas_mensais.groupby('dt_sale').agg({'qt_sale':'sum'}).reset_index(),\
              x='dt_sale', y='qt_sale')

fig.update_layout(xaxis_title="Período", yaxis_title="QTD. Vendida")

fig.show()

In [ ]:
#Agora analisando as series de cada uma das classes de produtos
fig = px.line(vendas_mensais.groupby([vendas_mensais.dt_sale, 'ds_product_line']).agg({'qt_sale':'sum'}).reset_index(),\
              x='dt_sale', y='qt_sale', color='ds_product_line')

fig.update_layout(xaxis_title="Período", yaxis_title="QTD. Vendida")

fig.show()

In [ ]:
fig = px.bar(vendas_mensais.groupby(['ds_sku']).agg({'qt_sale':'sum'}).reset_index().sort_values('qt_sale', ascending=False),\
              x='ds_sku', y='qt_sale', color='ds_sku')

fig.update_layout(xaxis_title="Período", yaxis_title="Quantidade", title='Quantidade de produtos vendidos')

fig.show()

In [ ]:
#Observa-se que os produtos mais vendidos são: Machina Metal, Max e Nautical Classic
fig = px.bar(vendas_mensais.groupby(['ds_sku']).agg({'vl_sale_price':'sum'}).reset_index().sort_values('vl_sale_price', ascending=False),\
              x='ds_sku', y='vl_sale_price', color='ds_sku')

fig.update_layout(xaxis_title="Período", yaxis_title="Vendas", title='Vendas Geradas')

fig.show()

_Quando olhamos as vendas o produto que gera maior capital é o Machina (TOP 1) seguido do Nautical Classic (TOP 3) por ultimo o Classic (TOP 5)_

_O preço unitário médio de cada classe apresenta flutu_

_Agora vamos realizar uma análise estatística dos dados_

In [ ]:
data = vendas_mensais.groupby(['dt_sale']).agg({'qt_sale':'sum'}).reset_index().qt_sale

# Calcule os valores de ACF e PACF
lags = 12
acf_values = acf(data, nlags=lags)
pacf_values = pacf(data, nlags=lags)

# Crie os gráficos ACF e PACF como scatter plots com linhas individuais
fig_acf = go.Figure(data=[
    go.Scatter(x=np.arange(len(acf_values)), y=acf_values, mode='markers', name='ACF'),
])

for i in range(len(acf_values)):
    fig_acf.add_shape(type='line',
                      x0=i, y0=acf_values[i],
                      x1=i, y1=0,
                      line=dict(color='black', width=1))
fig_acf.add_shape(
    type='line',
    x0=0, y0=-0.5,
    x1=len(acf_values), y1=-0.5,
    line=dict(color='red', width=1, dash='dash')
)

fig_acf.add_shape(
    type='line',
    x0=0, y0=0.5,
    x1=len(acf_values), y1=0.5,
    line=dict(color='red', width=1, dash='dash'))

fig_acf.update_layout(
    title="Autocorrelation Function (ACF)",
    xaxis_title="Lag",
    yaxis_title="ACF Value"
)

fig_pacf = go.Figure(data=[
    go.Scatter(x=np.arange(len(pacf_values)), y=pacf_values, mode='markers', name='PACF'),
])

for i in range(len(pacf_values)):
    fig_pacf.add_shape(type='line',
                       x0=i, y0=pacf_values[i],
                       x1=i, y1=0,
                       line=dict(color='black', width=1))
fig_pacf.add_shape(
    type='line',
    x0=0, y0=-0.25,
    x1=len(acf_values), y1=-0.25,
    line=dict(color='red', width=1, dash='dash')
)

fig_pacf.add_shape(
    type='line',
    x0=0, y0=0.25,
    x1=len(acf_values), y1=0.25,
    line=dict(color='red', width=1, dash='dash'))

fig_pacf.update_layout(
    title="Partial Autocorrelation Function (PACF)",
    xaxis_title="Lag",
    yaxis_title="PACF Value"
)

# Exiba os gráficos
fig_acf.show()
fig_pacf.show()


_Pelo grafico ACF podemos observar que nosso problema se trata de uma serie temporal não estacionária que possui autocorrelação. Além disso, ele reforça a ideia da sazonalidade_

_Como os spikes do gráfico PACF (Partial Autocorrelate Function) ultrapassam o intervalo demarcado, isso indica que a série possue autocorrelação e é não estacionária. Outro indicativo de que a série é autocorrelacionada é a presença de uma queda suave no gráfico ACF(Autocorrelate Function)_

In [ ]:
data = vendas_mensais.groupby(['dt_sale']).agg({'qt_sale':'sum'}).reset_index()
data = data.set_index('dt_sale')
decomposition = sm.tsa.seasonal_decompose(data['qt_sale'], model='additive', period=12)

fig = make_subplots(rows=4, cols=1, shared_xaxes=True)

# Criação dos gráficos separados para cada componente
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)

# Série temporal
fig.add_trace(go.Scatter(x=data.index, y=data['qt_sale'], name='Série Temporal'), row=1, col=1)
fig.update_yaxes(title_text='Quantidade Vendida', row=1, col=1)

# Sazonalidade
fig.add_trace(go.Scatter(x=data.index, y=decomposition.seasonal, name='Sazonalidade'), row=2, col=1)
fig.update_yaxes(title_text='Sazonalidade', row=2, col=1)

# Tendência
fig.add_trace(go.Scatter(x=data.index, y=decomposition.trend, name='Tendência'), row=3, col=1)
fig.update_yaxes(title_text='Tendência', row=3, col=1)

# Resíduos
fig.add_trace(go.Scatter(x=data.index, y=decomposition.resid,mode='markers', marker=dict(color='purple'), name='Resíduos'), row=4, col=1)
fig.update_yaxes(title_text='Resíduos', row=4, col=1)

fig.update_layout(height=800, title_text='Decomposição da Série Temporal')

fig.show()

_Nosso problema se baseia em uma série temporal com tendência positiva e observa-se a presença de uma sazonalidade. Em relação aos residuos, os mesmo apresentam média zero, distribuição proxima a normalidade, ainda precisa avaliar a questão da homocedasticidade_

## 3 - Limpeza e pré-processamento de dados

In [ ]:
# Importando os dados
df = pd.read_csv('/content/drive/tbl_vendas_mensais.csv', sep=';')

# Converte a coluna 'dt_sale' para tipo datetime.
# Joga fora as colunas não relevantes
# Converte a categoria de Sneaker para Sneakers
df['dt_sale'] = pd.to_datetime(df['dt_sale'], utc=True)
df.drop(columns=['ds_sku', 'ds_category', 'ds_brand_segment', 'ds_tecnology', 'vl_sale_price'], inplace=True)
df.loc[df['ds_product_line'] == 'Sneaker', 'ds_product_line'] = 'Sneakers'

# Faz a média nos valores de novembro de 2021
df_novembro_2021 = df[df['dt_sale'] == '2021-11']
df_novembro_2021 = df_novembro_2021.groupby(['id_sku', 'ds_product_line', 'dt_sale']).mean()
df_novembro_2021.reset_index(inplace=True)
df_novembro_2021 = df_novembro_2021.set_index('dt_sale')

# Remove os valores antigos de novembro de 2021 e adiciona os novos
df.drop(index=df[df['dt_sale'] == '2021-11'].index, inplace=True)
df.set_index('dt_sale', drop=True, inplace=True)
df = pd.concat([df, df_novembro_2021])
df.drop(columns='id_sku', inplace=True)
df.sort_index(inplace=True)

# Retorna o dataset final. Para cada produto('ds_product_line') temos a quantidade total de vendas.
df = df.groupby(['dt_sale', 'ds_product_line']).sum()
df.reset_index(inplace=True)
df

,dt_sale,ds_product_line,qt_sale
0,2019-01-01 00:00:00+00:00,Boot,191839.0
1,2019-01-01 00:00:00+00:00,Sandals,311140.0
2,2019-01-01 00:00:00+00:00,Shoes,575501.0
3,2019-01-01 00:00:00+00:00,Sneakers,279316.0
4,2019-02-01 00:00:00+00:00,Boot,214879.0
...,...,...,...
199,2023-02-01 00:00:00+00:00,Sneakers,511301.0
200,2023-03-01 00:00:00+00:00,Boot,184398.0
201,2023-03-01 00:00:00+00:00,Sandals,481353.0
202,2023-03-01 00:00:00+00:00,Shoes,812154.0


# **4 - Implementação dos Modelos**

## **Rede Neural Recorrente (arquitetura LSTM)**


In [ ]:
df = df.loc[:, ['dt_sale', 'ds_product_line', 'qt_sale']]

tbl = df.groupby(['dt_sale', 'ds_product_line']).sum()
tbl = tbl.sort_values(by='dt_sale')
tbl = tbl.reset_index()

#Separando o Df de cada categoria
df_Boot = tbl[tbl['ds_product_line'] == 'Boot']
df_Sandals = tbl[tbl['ds_product_line'] == 'Sandals']
df_Shoes = tbl[tbl['ds_product_line'] == 'Shoes']
df_Sneakers = tbl[tbl['ds_product_line'] == 'Sneakers']

In [ ]:
#Função que separa os atributos previsores para treinamento da rede neural
def Sepacao_previsores(df):
    df_treino = df.iloc[:47,:]

    #Escalonando
    df_treino = df_treino['qt_sale'].values
    df_treino = df_treino.reshape(-1, 1)
    escaler = StandardScaler()
    df_treino_escalonada = escaler.fit_transform(df_treino)

    previsores = []
    qt_real = []

    #Para a previsão de um dia, nesse caso, são necessários analisar os 12 meses anteriores
    for i in range(12, len(df_treino)):
        previsores.append(df_treino_escalonada[i - 12:i, 0])
        qt_real.append(df_treino_escalonada[i, 0])

    #Reformatação para a forma requerida pelo KERAS
    previsores, qt_real = np.array(previsores), np.array(qt_real)
    previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))
    return previsores, qt_real, escaler

In [ ]:
#Função que aplica a rede neural
def RNN(previsores, qt_real):
    #Arquitetura da rede neural
    regressor = Sequential()
    regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1], 1)))
    regressor.add(Dropout(0.3)) #Irá zerar 30% das entradas (Ajuda a previnir overfitting)

    regressor.add(LSTM(units=40, return_sequences=True))
    regressor.add(Dropout(0.3))

    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.3))

    regressor.add(LSTM(units=60, return_sequences=False))
    regressor.add(Dropout(0.2))

    #Camada de saída
    regressor.add(Dense(units= 1, activation='linear'))

    regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics = ['mean_absolute_error'])

    #mc = ModelCheckpoint('Best_weights.h5', save_best_only=True, monitor='loss', verbose=1) #Salva os melhores pesos da RN

    regressor.fit(previsores, qt_real, epochs=200, batch_size=12)
    return regressor

In [ ]:
#função que a partir do modelo treinado cria por meio de condicionais as colunas m1, m2 e m3
def Previsões(df, regressor, escaler):
    m1 = []
    m2 = []
    m3 = []
    #Ciclo que vai de setembro até fevereiro
    for i in range(45, 51):
        #Previsão de Setembro
        if i == 45:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 2:
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0) #Retro alimentação, excluindo o primeiro...
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0) #E adicionando a previsão com ultimo
        #Previsão de Outubro
        if i == 46:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))
                if a == 2: #M - 3
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Novembro
        if i == 47:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))
                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))
                if a == 2: #M - 3
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Dezembro
        if i == 48:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))
                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))
                if a == 2: #M - 3
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Janeiro
        if i == 49:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))
                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))

                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Fevereiro
        if i == 50:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))

                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)

    Product = df.iloc[47:]

    Product['Previsão_M1'] = m1
    Product['Previsão_M2'] = m2
    Product['Previsão_M3'] = m3
    return Product

In [ ]:
#Chamado das funções e geração das tabelas de previsão
previsores, qt_real, escaler = Sepacao_previsores(df_Boot)
Boot = Previsões(df_Boot, RNN(previsores, qt_real), escaler)

previsores, qt_real, escaler = Sepacao_previsores(df_Sandals)
Sandals = Previsões(df_Sandals, RNN(previsores, qt_real), escaler)

previsores, qt_real, escaler = Sepacao_previsores(df_Shoes)
Shoes = Previsões(df_Shoes, RNN(previsores, qt_real), escaler)

previsores, qt_real, escaler = Sepacao_previsores(df_Sneakers)
Sneakers = Previsões(df_Sneakers, RNN(previsores, qt_real), escaler)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - loss: 0.9253 - mean_absolute_error: 0.8218
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1.0469 - mean_absolute_error: 0.8696 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1.0374 - mean_absolute_error: 0.9156
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.9516 - mean_absolute_error: 0.8591
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 1.0270 - mean_absolute_error: 0.8732
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8352 - mean_absolute_error: 0.8014
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.8280 - mean_absolute_error: 0.7825
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.9486 - mean_absolute_error: 0.8798
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.7391 - mean_absolute_error: 0.7382
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.9456 - mean_absolute_error: 0.8546
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/st

<ipython-input-40-d5464244d68c>:25: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

<ipython-input-40-d5464244d68c>:45: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

<ipython-input-40-d5464244d68c>:48: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



...

<ipython-input-40-d5464244d68c>:120: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

<ipython-input-40-d5464244d68c>:123: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

<ipython-input-40-d5464244d68c>:144: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


<ipython-input-40-d5464244d68c>:151: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-40-d5464244d68c>:152: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-40-d5464244d68c>:153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
Boot

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
190,2022-12-01 00:00:00+00:00,Boot,263267.0,277261.093750,277317.968750,277183.000000
192,2023-01-01 00:00:00+00:00,Boot,223118.0,221663.625000,221656.906250,221560.671875
196,2023-02-01 00:00:00+00:00,Boot,215163.0,222601.328125,222602.421875,222625.203125
201,2023-03-01 00:00:00+00:00,Boot,184398.0,229673.515625,229622.812500,229632.781250


In [ ]:
Sandals

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
191,2022-12-01 00:00:00+00:00,Sandals,661730.0,527825.37500,528523.93750,529636.12500
193,2023-01-01 00:00:00+00:00,Sandals,576581.0,504029.15625,505601.53125,507146.68750
197,2023-02-01 00:00:00+00:00,Sandals,573049.0,507219.46875,507678.28125,510601.50000
202,2023-03-01 00:00:00+00:00,Sandals,481353.0,524136.31250,524026.71875,523872.34375


In [ ]:
Shoes

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
189,2022-12-01 00:00:00+00:00,Shoes,1227099.0,1.181798e+06,1.181666e+06,1.181667e+06
194,2023-01-01 00:00:00+00:00,Shoes,1052338.0,1.040049e+06,1.040922e+06,1.044943e+06
198,2023-02-01 00:00:00+00:00,Shoes,944811.0,7.906188e+05,7.907215e+05,7.933022e+05
200,2023-03-01 00:00:00+00:00,Shoes,812154.0,8.096997e+05,8.092027e+05,8.092858e+05


In [ ]:
Sneakers

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
188,2022-12-01 00:00:00+00:00,Sneakers,564820.0,570341.18750,570351.31250,570295.50000
195,2023-01-01 00:00:00+00:00,Sneakers,521048.0,460967.12500,460985.28125,460920.53125
199,2023-02-01 00:00:00+00:00,Sneakers,511301.0,436176.06250,435869.90625,435846.81250
203,2023-03-01 00:00:00+00:00,Sneakers,441090.0,440149.34375,439706.28125,439585.46875


In [ ]:
def calculate_mape(actual, predicted):
  return np.mean(np.abs((actual - predicted) / actual)) * 100


dfs = [Boot, Sandals, Shoes, Sneakers]
results = []

for df in dfs:
  for col in ['Previsão_M1', 'Previsão_M2', 'Previsão_M3']:
    mape = calculate_mape(df['qt_sale'], df[col])
    results.append({'Product': df['ds_product_line'].iloc[0], 'Month': col, 'MAPE': mape})

result_df = pd.DataFrame(results)
display(result_df[['Product', 'Month', 'MAPE']])


,Product,Month,MAPE
0,Boot,Previsão_M1,8.494402
1,Boot,Previsão_M2,8.493809
2,Boot,Previsão_M3,8.495773
3,Sandals,Previsão_M1,13.298595
4,Sandals,Previsão_M2,13.178319
5,Sandals,Previsão_M3,12.933757
6,Shoes,Previsão_M1,5.370404
7,Shoes,Previsão_M2,5.364929
8,Shoes,Previsão_M3,5.198539
9,Sneakers,Previsão_M1,6.853611


In [ ]:
def calculate_mse(actual, predicted):
  return np.mean(np.square(actual - predicted))


dfs = [Boot, Sandals, Shoes, Sneakers]
results = []

for df in dfs:
  for col in ['Previsão_M1', 'Previsão_M2', 'Previsão_M3']:
    mse = calculate_mse(df['qt_sale'], df[col])
    results.append({'Product': df['ds_product_line'].iloc[0], 'Month': col, 'MSE': mse})

result_df = pd.DataFrame(results)
display(result_df[['Product', 'Month', 'MSE']])


,Product,Month,MSE
0,Boot,Previsão_M1,5.757877e+08
1,Boot,Previsão_M2,5.750483e+08
2,Boot,Previsão_M3,5.744876e+08
3,Sandals,Previsão_M1,7.339539e+09
4,Sandals,Previsão_M2,7.219079e+09
5,Sandals,Previsão_M3,6.994375e+09
6,Shoes,Previsão_M1,6.496122e+09
7,Shoes,Previsão_M2,6.486692e+09
8,Shoes,Previsão_M3,6.270475e+09
9,Sneakers,Previsão_M1,2.321209e+09


In [ ]:
def calculate_iae(actual, predicted):
  return np.sum(np.abs(actual - predicted))

dfs = [Boot, Sandals, Shoes, Sneakers]
results = []

for df in dfs:
  for col in ['Previsão_M1', 'Previsão_M2', 'Previsão_M3']:
    iae = calculate_iae(df['qt_sale'], df[col])
    results.append({'Product': df['ds_product_line'].iloc[0], 'Month': col, 'IAE': iae})

result_df = pd.DataFrame(results)
display(result_df[['Product', 'Month', 'IAE']])


,Product,Month,IAE
0,Boot,Previsão_M1,68162.312500
1,Boot,Previsão_M2,68176.296875
2,Boot,Previsão_M3,68170.312500
3,Sandals,Previsão_M1,315069.312500
4,Sandals,Previsão_M2,312229.968750
5,Sandals,Previsão_M3,306495.031250
6,Shoes,Previsão_M1,214236.750000
7,Shoes,Previsão_M2,213889.812500
8,Shoes,Previsão_M3,207204.000000
9,Sneakers,Previsão_M1,141667.656250


In [ ]:
def calculate_mae(actual, predicted):
  return np.mean(np.abs(actual - predicted))

dfs = [Boot, Sandals, Shoes, Sneakers]
results = []

for df in dfs:
  for col in ['Previsão_M1', 'Previsão_M2', 'Previsão_M3']:
    mae = calculate_mae(df['qt_sale'], df[col])
    results.append({'Product': df['ds_product_line'].iloc[0], 'Month': col, 'MAE': mae})

result_df = pd.DataFrame(results)
display(result_df[['Product', 'Month', 'MAE']])


,Product,Month,MAE
0,Boot,Previsão_M1,17040.578125
1,Boot,Previsão_M2,17044.074219
2,Boot,Previsão_M3,17042.578125
3,Sandals,Previsão_M1,78767.328125
4,Sandals,Previsão_M2,78057.492188
5,Sandals,Previsão_M3,76623.757812
6,Shoes,Previsão_M1,53559.187500
7,Shoes,Previsão_M2,53472.453125
8,Shoes,Previsão_M3,51801.000000
9,Sneakers,Previsão_M1,35416.914062
